In [ ]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.backend import sigmoid
from sklearn.metrics import confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import precision_score, make_scorer
import matplotlib.pyplot as plt
from math import sqrt
import threading
from multiprocessing.pool import ThreadPool
import time
import multiprocessing
import traceback
import tensorflow as tf


In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [ ]:
def reshape_data(x_train, x_test, y_train, y_test,units = 30):
    my_x_train = list()
    my_y_train = list()
    my_x_test = list()
    my_y_test = list()
    for i in range(x_train.shape[0]-units):
        my_x_train.append(x_train.iloc[i:i+units,:])
        my_y_train.append(y_train.iloc[i+units,])
    
    my_x_train = np.array(my_x_train)
    my_x_train = np.reshape(my_x_train,(my_x_train.shape[0],my_x_train.shape[1],my_x_train.shape[2]))
    
    my_y_train = np.array(my_y_train)
    my_y_train = np.reshape(my_y_train,(my_y_train.shape[0],1))
    
    for i in range(x_test.shape[0]-units):
        my_x_test.append(x_test.iloc[i:i+units,:])
        my_y_test.append(y_test.iloc[i+units,])
        
    my_x_test = np.array(my_x_test)
    my_x_test = np.reshape(my_x_test,(my_x_test.shape[0],my_x_test.shape[1],my_x_test.shape[2]))
    
    my_y_test = np.array(my_y_test)
    my_y_test = np.reshape(my_y_test,(my_y_test.shape[0],1))
    
    return (my_x_train, my_x_test, my_y_train, my_y_test)

In [ ]:
def error_metrics(y_true, y_pred):
    rmse = sqrt(metrics.mean_squared_error(y_true, y_pred))
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse}

In [ ]:
def create_confusion_matrix(y_true,y_pred):
    
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [ ]:
def build_lstm(input_shape,optimizer,activation,dropout_rate,neurons,layers):
    model = Sequential()
    model.add(LSTM(neurons,return_sequences=True,input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(neurons))
    model.add(Dropout(dropout_rate))
    for _ in range(layers):
        model.add(Dense(neurons))
        model.add(Dropout(dropout_rate))
    model.add(Dense(units=1,activation = activation))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[tf.metrics.Precision()])
    return model

In [ ]:
def split_dataset(X,Y,t):
    tr = int(len(X)*t)
    tt = len(X) - tr
    xtr = X[:tr]
    xtt = X[tr:tr+tt]
    ytr = Y[:tr]
    ytt = Y[tr:tr+tt]
    return (xtr,xtt,ytr,ytt)

In [ ]:
def run_process_model(optimizer,activation,dropout_rate,neurons,layers,batch_size,epochs):
    try:
        print(optimizer,activation,dropout_rate,neurons,layers,batch_size,epochs)
        global x_train, x_test, y_train, y_test
        input_shape = (x_train.shape[1],x_train.shape[2])
        model =  KerasClassifier(build_lstm,input_shape=input_shape,optimizer=optimizer,activation=activation,dropout_rate=dropout_rate,neurons=neurons,layers=layers)
        history = model.fit(x_train,y_train,epochs = int(epochs) ,batch_size = int(batch_size), validation_data = (x_test,y_test),shuffle = False,verbose=0)
        y_pred = model.predict(x_test)
        result = {}
        confusion = create_confusion_matrix(y_test,y_pred)
        result.update(confusion)
        result.update({'epochs':epochs,'batch_size':batch_size})
        result.update({'optimizer':optimizer,'activation':activation,'dropout_rate':dropout_rate,'neurons':neurons,'layers':layers})
        return result
    except:
        traceback.print_exc()
        return []

In [ ]:
def compute():    
    best_epochs,best_batch_size,best_layer,best_neurons,best_dropout_rate,best_activation,best_optimizer = 25,25,2,32,0.2,'sigmoid','adam'
    prev = 0
    for b in [25,32,50,75,100]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=0.2,neurons=32,layers=2,batch_size=b,epochs=25)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_batch_size = res['batch_size']
        else:
            break
            
    for e in [50,75,100]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=0.2,neurons=32,layers=2,batch_size=best_batch_size,epochs=e)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_epochs = res['epochs']
        else:
            break
            
    for l in [3,4]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=0.2,neurons=32,layers=l,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_layer = res['layers']
        else:
            break

    for n in [64]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=0.2,neurons=n,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_neurons = res['neurons']
        else:
            break
    
    for d in [0.1,0]:
        res = run_process_model(optimizer = 'adam',activation='sigmoid',dropout_rate=d,neurons=best_neurons,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_dropout_rate = res['dropout_rate']
        else:
            break
            
    best_activation = 'sigmoid'
    
    for o in ['sgd']:
        res = run_process_model(optimizer = o,activation=best_activation,dropout_rate=best_dropout_rate,neurons=best_neurons,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
        print(prev,res['precision'])
        if (res['precision'] > prev):
            prev = res['precision']
            best_optimizer = res['optimizer']
        else:
            break
    print(best_epochs,best_batch_size,best_layer,best_neurons,best_dropout_rate,best_activation,best_optimizer)
    return (best_epochs,best_batch_size,best_layer,best_neurons,best_dropout_rate,best_activation,best_optimizer)

In [ ]:
security_codes = list()
for filename in os.listdir("../input/newdata/grstocks"):
    security_codes.append(filename[2:-4])

In [ ]:
# for _,row in top.iterrows():
for name in security_codes:
    try:
        df = pd.read_csv("../input/newdata/grstocks/"+"gr"+str(name)+".csv")
        df = df.iloc[::-1].reset_index(drop=True)
        df = pre_process_data(df,60)
        df,column = dependent_column(df,"Next Day Close Price GR")
        myresult = list()
        for t in [0.001,0.002,0.003,0.004,0.005]:
            print(name,t)
            try:
                df["Target"] = df[column].apply(lambda x : 1 if x >= t else 0)
                X = df.drop(columns=["Target",column])
                Y = df["Target"]
                x_train, x_test, y_train, y_test = split_dataset(X, Y,0.70)
                (x_train, x_test, y_train, y_test) = reshape_data(x_train, x_test, y_train, y_test)
                best_epochs,best_batch_size,best_layer,best_neurons,best_dropout_rate,best_activation,best_optimizer = compute()
                result = run_process_model(optimizer = best_optimizer,activation=best_activation,dropout_rate=best_dropout_rate,neurons=best_neurons,layers=best_layer,batch_size=best_batch_size,epochs=best_epochs)
                result.update({"rate_of_growth":t})
                myresult.append(result)
            except:
                traceback.print_exc()
        pd.DataFrame(myresult).to_csv("rnn_"+str(name)+".csv",index=None)
    except:
        traceback.print_exc()